In [1]:
from MaptoMask import *
from graphinference import *

### Reading probability map images of cell body and neurite from directory

In [2]:
c_data_directory = "/Users/behnaz/NCADD/paper_pipeline_v1/img/save_path_c_map/"
c_all_tiff_files = []
N = 0
for file in os.listdir(c_data_directory):
    if file.startswith("pre"):
        if file.endswith(".tif"):
            c_all_tiff_files.append(file)
            N+=1
        
c_all_names = [] 
for file in c_all_tiff_files: 
    c_all_names.append(file[:])  

c_all_names = sorted(c_all_names, key = lambda x: (str(x[:])))



n_data_directory = "/Users/behnaz/NCADD/paper_pipeline_v1/img/save_path_n_map/"
n_all_tiff_files = []
N = 0
for file in os.listdir(n_data_directory):
    if file.startswith("pre"):
        if file.endswith(".tif"):
            n_all_tiff_files.append(file)
            N+=1
        
n_all_names = [] 
for file in n_all_tiff_files: 
    n_all_names.append(file[:])  

n_all_names = sorted(n_all_names, key = lambda x: (str(x[:])))

## This section comprises several steps:

1. Mapping to binary mask images.
2. Calculating the number of cells and cell body area per field of view (FOV or image).
3. Inferring structural graphs from the two mask images.
4. Inferring neurite length from the weight matrix.
5. Obtaining the adjacency matrix from the weight matrix.
6. Saving the results in the specified format.


In [ ]:
save_path_c = "/Users/behnaz/NCADD/paper_pipeline_v1/img/save_path_c"
save_path_n = "/Users/behnaz/NCADD/paper_pipeline_v1/img/save_path_n"
save_adj = "/Users/behnaz/NCADD/paper_pipeline_v1/adj"

cell_count = []
cell_areas = []
Neurite_length = {}

groups = {'05DIV':[],'09DIV':[],'12DIV':[],'17DIV':[]} # day in vitro (DIV) ID

for i in range(len(n_all_names)):
    
    # Extract the ID from the file name and append the file to the appropriate group
    file = n_all_names[i][:-16]
    
    # Extract the ID from the file name and append the file to the appropriate group
    id = file.split('_')[1]
    if id in groups:
        print(id,file)
        groups[id].append(n_all_names[i][:-16])
    
    #Read the image
    cmap_img = img_read(c_data_directory, c_all_names[i])
    
    nmap_img = img_read(n_data_directory, n_all_names[i])
    
    
    # Get binary mask images with the given threshold
    cell_mask, neurite_mask, cell_num = binary_masks(cmap_img, nmap_img)
    
    # Cell density in the given FOV
    cell_count.append(int(cell_num/0.87)) # # of cells/mm2
    
    
    # Cell body area 
    regions = measure.regionprops(label_image=cell_mask)
    cell_areas.append([convert_pixel2_to_um2(region.area) for region in regions])
    
    
    # infer structural graph from the two masks (cell_mask, neurite_mask)
    vertices, weights, paths, paths_matrix = infer_structural_graph(cell_mask, neurite_mask, find_paths=True)
    adj_mat = weights > 0
    
    # Create the full path including the file name and extension
    full_path = os.path.join(save_adj, f"{n_all_names[i][:-16]}.npy")
    # Save the NumPy array to the specified path
    np.save(full_path, adj_mat)
    
    
    #Compute neurite length
    neurite_length = np.reciprocal(weights)
    neurite_length[np.isinf(neurite_length)] = 0
    up_neurite_length = np.triu(neurite_length,1)
    non_zero_mask = up_neurite_length != 0
    neurite_length = convert_pixel_to_um(up_neurite_length[non_zero_mask])
    
    # dictionary of the neurite length based on the day in vitro id 
    if id in Neurite_length:
        Neurite_length[id].extend(neurite_length)
    else:
        Neurite_length[id] = list(neurite_length)
    
    # Create the full path to save the image
    full_path_cmap = os.path.join(save_path_c, id+'_cell_body_mask.tif')
    # Create the full path to save the image
    full_path_nmap = os.path.join(save_path_n, id+'_neurite_mask.tif')

    # Save the image to the specified path
    io.imsave(full_path_cmap, cell_mask)
    io.imsave(full_path_nmap, neurite_mask)

### Save dicts

In [ ]:
save_dict = "/Users/behnaz/NCADD/paper_pipeline_v1/dicts"
# Create a dictionary to store the arrays for each group
group_data_cell_count = {group: [] for group in groups}
group_data_cell_areas = {group: [] for group in groups}


# Iterate through the files and append the corresponding array value to the group data
for i in range(len(cell_areas)):
    file = n_all_names[i][:-16]
    id = file.split('_')[1]
    print(id)
    group_data_cell_count[id].append(cell_count[i])
    group_data_cell_areas[id].append(cell_areas[i])
    
# Create the full path including the file name
file_name = 'cell_areas.json'
full_path = os.path.join(save_dict, file_name)

# Save the JSON data to the specified path
with open(full_path, 'w') as file:
    json.dump(group_data_cell_areas, file)    
    
    
# Create the full path including the file name
file_name = 'neurite_length.json'
full_path = os.path.join(save_dict, file_name)

# Save the JSON data to the specified path
with open(full_path, 'w') as file:
    json.dump(Neurite_length, file) 

    
# Create the full path including the file name
file_name = 'cell_count.json'
full_path = os.path.join(save_dict, file_name)

# Save the JSON data to the specified path
with open(full_path, 'w') as file:
    json.dump(group_data_cell_count, file, default=convert_np_types) 